In [1]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
train_data = datasets.MNIST(
    root='data',
    train=True,
    transform=ToTensor(),
    download=True
)

test_data = datasets.MNIST(
    root='data',
    train=False,
    transform=ToTensor(),
    download=True
)

100%|██████████| 9912422/9912422 [00:06<00:00, 1597929.32it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 41915464.99it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 2684693.55it/s]


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 15438029.80it/s]

Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw


In [28]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [29]:
test_data.data.size()

torch.Size([10000, 28, 28])

In [23]:
from torch.utils.data import DataLoader

loaders = {
    'train': DataLoader(
        train_data, 
        batch_size=100,
        shuffle=True,
        num_workers=1),
    
    'test_loader': DataLoader(
        test_data, 
        batch_size=100,
        shuffle=True,
        num_workers=1),
}   